<a href="https://colab.research.google.com/github/ClaireZixiWang/learn2cut/blob/main/Project_policy_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## See README.md file for further details about the project and the environment.

### State-Action Description

### State
State s is an array with give components

* s[0]:  constraint matrix $A$of the current LP ($\max  -c^Tx \text{ s.t. }Ax \le  b$) . Dimension is $m \times n$. See by printing s[0].shape. Here $n$ is the (fixed) number of variables. For instances of size 60 by 60 used in the above command, $n$ will remain fixed as 60. And $m$ is the current number of constraints. Initially, $m$ is to the number of constraints in the IP instance. (For instances generated with --num-c=60, $m$ is 60 at the first step).  But $m$ will increase by one in every step of the episode as one new constraint (cut) is added on taking an action.
* s[1]: rhs $b$ for the current LP ($Ax\le b$). Dimension same as the number $m$ in matrix A.
* s[2]: coefficient vector $c$ from the LP objective ($-c^Tx$). Dimension same as the number of variables, i.e., $n$.
* s[3],  s[4]: Gomory cuts available in the current round of Gomory's cutting plane algorithm. Each cut $i$ is of the form $D_i x\le d_i$.   s[3] gives the matrix $D$ (of dimension $k \times n$) of cuts and s[4] gives the rhs $d$ (of dimension $k$). The number of cuts $k$ available in each round changes, you can find it out by printing the size of last component of state, i.e., s[4].size or s[-1].size.

### Actions
There are k=s[4].size actions available in each state $s$, with $i^{th}$ action corresponding to the $i^{th}$ cut with inequality $D_i x\le d_i$ in $s[3], s[4]$.

### ***QUESTIONS***:
1. By "current" LP, you mean the LP that the agent was running in the last state? As in, the LP with all the added constraints? 
  * ==> I think so.
1. What do you mean Gomory cuts *available*? As in, after doing Simplex methods, the *variables* that you can choose to cut?
  * Yes I think so.
2. Isn't the number of variables (n) changing? in the C-G cutting plane method?
  * No, as the spec says, **$n$ is the fixed number of variables**.
  * If you look that cuttng plane lecture notes, you can see that after each step, the dummy variable is not added in the constraint. They are merely there for the sake of the LP solver (simplex method), but not really relevant for us.
    * This is not correct, I think they are still very much relevant, it's just that I think among the 60 variables a lot of them are space holders for dummy variables so that our $n$ is fixed, so that we don't have to worry about using LSTM. Since each time the sequence [a, b] will be of size n+1. And we can just use a fixed-input-size network to do that.
    * But still need to verify with the TA about the place holder understanding.
3. dimension of s[3] and s[4]? Where is the "available all" stored? In which dimension?
  * Each row of D is an "available cut". Therefore each $D_i x\le d_i$ is an "available" cut in CG method solved from the simplex method.
4. pointing towards the slides: why does the number of constraints m increase 1 in each step, if you can choose *multiple* cuts in one step? (OR in the algorithm we just choose one cut each time? or is that a more vanilla version to start, but to expand on multiple cuts a time later?)
5. What do you mean by each "instance"?

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
!pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com


In [33]:
!pip install wandb -qqq

In [34]:
%cd drive/MyDrive/IEOR_RL/Project_learn2cut
%pwd

[Errno 2] No such file or directory: 'drive/MyDrive/IEOR_RL/Project_learn2cut'
/content/drive/MyDrive/IEOR_RL/Project_learn2cut


'/content/drive/MyDrive/IEOR_RL/Project_learn2cut'

In [35]:
import torch
import numpy as np
import random
import time

In [36]:
model = torch.nn.Sequential(
            torch.nn.Linear(4, 40),
            torch.nn.ReLU(),
            torch.nn.Linear(40, 20), 
            torch.nn.ReLU(),
            torch.nn.Linear(20, 10)
        )

datapoint = torch.FloatTensor([
                               [[1,2,3,4],
                                [2,3,4,5]],
                               [[3,4,5,6],
                                [4,5,6,7]]
                              ])


In [37]:
model(datapoint)

tensor([[[-0.5739, -0.0712, -0.1939, -0.1789,  0.2275,  0.0254, -0.4184,
           0.4109,  0.1992,  0.0089],
         [-0.6962, -0.0828, -0.1940, -0.2345,  0.2921,  0.0213, -0.4829,
           0.5324,  0.2761,  0.0707]],

        [[-0.8191, -0.0946, -0.1938, -0.2877,  0.3556,  0.0170, -0.5495,
           0.6553,  0.3525,  0.1308],
         [-0.9420, -0.1064, -0.1935, -0.3409,  0.4190,  0.0126, -0.6160,
           0.7781,  0.4288,  0.1908]]], grad_fn=<AddBackward0>)

In [38]:
## Code for Policy Model

class Policy(object):

    # inputsize = n+1 = 61
    def __init__(self, lr, input_size, attention_size=10, temperature=1) -> None:
        super().__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 40),
            # torch.nn.Sigmoid(),
            torch.nn.ReLU(),
            torch.nn.Linear(40, 30), 
            # torch.nn.Sigmoid(),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20), 
            # torch.nn.Sigmoid(),
            torch.nn.ReLU(),
            torch.nn.Linear(20, attention_size)
        )

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # RECORD HYPER-PARAMS
        self.input_size = input_size
        self.attention_size = attention_size
        self.temperature = temperature

    def compute_logits_big_batch(self, obs_matrix, act_matrix, batchsize):
        '''
        Function that takes in a batch of observations and computes the action logits for each of observations in this batch
        Args: obs_matrix: np.array(m * batchsize, n+1) # TODO maybe change this to tensor
              act_matrix: np.array(k * batchsize, n+1)
        Return: batch_logit: tensor(batchsize, k)
        '''

        # Get the batch result

        # transform to tensor
        obs_attention = self.model(torch.FloatTensor(obs_matrix)) # tensor(m * batchsize, u)
        act_attention = self.model(torch.FloatTensor(act_matrix)) # tensor(k * batchsize, u)

        assert obs_attention.shape == (obs_matrix.shape[0], self.attention_size)
        assert act_attention.shape == (act_matrix.shape[0], self.attention_size)

        # split a batch of output of size tensor(m * batchsize, u) into (batchsize, m, u)
        batch_obs_output = torch.reshape(obs_attention, (batchsize, obs_attention.shape[0]/batchsize, obs_attention.shape[1]))
        
        # split a batch of output of size tensor(k * batchsize, u) into (batchsize, k, u)
        batch_act_output = torch.reshape(act_attention, (batchsize, act_attention.shape[0]/batchsize, act_attention.shape[1]))

        # To do batch matrix multiplication, transpose (batchsize, k, u) into (batchsize, u, k)
        # (batchsize, m, u) @ (batchsize, u, k) =  (batchsize, m, k)
        # (batchsize, m, k) == mean across all observation ==> (batchsize, k)
        batch_logit = torch.bmm(batch_obs_output, batch_act_output.transpose(0, 2, 1)).mean(dim=1)

        assert batch_logit.shape == (batchsize, act_matrix.shape[0]/batchsize)

        return batch_logit

    def compute_batch_selected_prob(self, batch_probs):
        '''
        Function that takes in a batch of probabilities and return the max probability for each "datapoint" in the batch
        Args:    batch_logit: tensor(batchsize, k)
        Return:  batch_selected_prob: tensor(batchsize,)
        '''
        # TODO

        return


    def compute_one_step_logits(self, obs, act):
        '''
        Function that takes in ONE observation and action space, computes the action logits
        Args:   obs_matrix: np.array(m, n+1)
                act_matrix: np.array(k, n+1)
        Return: logit: tensor(k)
        '''

        obs_attention = self.model(torch.FloatTensor(obs)) #-> (m, 10)
        act_attention = self.model(torch.FloatTensor(act)) #-> (k, 10)
        # print("DEBUGGING: obs_attention looks like:", obs_attention)
        # print("DEBUGGING: act_attention looks like:", act_attention)

        # attention matrix multiplication & mean to get the score
        logits = torch.mm(obs_attention, act_attention.transpose(1, 0)).mean(dim=0)
        # print("DEBUGGING: logits looks like:", logits)

        # print("DEBUGGING: act.shape =", act_attention.shape)
        # print("DEBUGGING: logits.shape =", logits.shape)
        assert logits.shape[0] == act_attention.shape[0]

        return logits

    def predict_prob(self, obs, act):
        # Function that uses softmax to transform logits to probabilities
        # TODO: What shape should obs and act take?
        # Args:   obs_matrix: np.array(m, n+1)
        #         act_matrix: np.array(k, n+1)
        # Return: probs: tensor(k)

        logits = self.compute_one_step_logits(obs, act)

        # TODO: make temperature a argument in the function, so that it's depended on the steps
        probs = torch.nn.functional.softmax(logits/self.temperature, dim=0)
        return probs

    def selected_prob(self, probs, action):


        # TODO: do I need this function?
        one_hot = torch.zeros()
        return probs.max()

    def choose_action(self, obs, act):

        # TODO: is this returning a number?
        return torch.argmax(self.predict_prob(obs, act)).item()


    def train(self, obs_matrix, act_matrix, actions, Qs):
        """
        Args: obs_matrix: np.array(batchsize * m, n+1) => changed to [np.array(m, n+1)] * batchsize, note that m are varied!
              act_matrix: np.array(batchsize * k, n+1)  
              actions: => [[action number]] * batchsize      
              Qs: np.array(batchsize, )
        """
        # Convert numpy array to tensor

        # use compute_batch_prob to compute the batch logits for every datapoint in batch.
        # use compute_batch_selected_prob the compute the max probabilty for every datapoint in batch.
        start = time.time()
        print("DEBUGGING: I'm inside the training now!")
        
        Qs = torch.FloatTensor(Qs)


        # Try using a for loop first, see whether it really cost too much time & whether it works at all
        prob_selected = torch.zeros(len(actions))
        for i in range(len(obs_matrix)):
            #TODO: adjust the temperatures based on trajectory steps
            probs = self.predict_prob(obs_matrix[i], act_matrix[i])
            one_hot = torch.zeros_like(probs)
            one_hot[actions[i][0]] = 1
            prob_selected[i] = torch.sum(probs * one_hot)


        # For robustness add in noise for prob_selected # TODO: why do this?

        # define loss function as in lab 4
        # TODO define loss function as described in the text above
        loss = - (torch.sum(Qs * torch.log(prob_selected)) / (len(obs_matrix) + 1))
        print("DEBUGGING: the loss =", loss)

        print("DEBUGGING: BEFORE the backward pass, the gradients of the network looks like:")
        print("   First layer:")
        print(policy.model[0].weight.grad)
        print("   Last layer:")
        print(policy.model[6].weight.grad)

        # backward pass
        self.optimizer.zero_grad()
        loss.backward()


        print("DEBUGGING: AFTER the backward pass, the gradients of the network looks like:")
        print("   First layer:")
        print(policy.model[0].weight.grad)
        print("   Last layer:")
        print(policy.model[6].weight.grad)

        # step
        self.optimizer.step()


        print("DEBUGGING: training for one iteration takes %f min:" % ((time.time() - start)/60))


        # return detached loss (why?)
        return loss.detach().cpu().data.numpy()





In [39]:
# from torchsummary import summary
# summary(policy.model, (61,))
# print(policy.model)

In [40]:
# print(policy.model[6].weight.grad)

In [41]:
# MLP model for policy model:
#   model.forward
#   model.train --> What is in this function? what are the function arguments?
# Baseline function b(s) ==> Okay maybe we stil need the V model as a proper baseline


# Q value model ==> Discard this right now, just do vanilla policy gradient
#   Can I just use the one in Lab4? What does it mean? what does the states and actions mean? --> Print out the s, r to check
#   What is a Q-value in our set-up?
#   How do I used this? 
#   (What's the baseline function??)

In [42]:
# import gymenv_v2
from gymenv_v2 import make_multiple_env


import wandb
wandb.login()
run=wandb.init(project="finalproject", entity="ieor4575-spring2022", tags=["training-easy"])
#run=wandb.init(project="finalproject", entity="ieor-4575", tags=["training-hard"])
#run=wandb.init(project="finalproject", entity="ieor-4575", tags=["test"])

### TRAINING

# Setup: You may generate your own instances on which you train the cutting agent.
custom_config = {
    "load_dir"        : 'instances/randomip_n60_m60',   # this is the location of the randomly generated instances (you may specify a different directory)
    "idx_list"        : list(range(20)),                # take the first 20 instances from the directory
    "timelimit"       : 50,                             # the maximum horizon length is 50
    "reward_type"     : 'obj'                           # DO NOT CHANGE reward_type
}

# Easy Setup: Use the following environment settings. We will evaluate your agent with the same easy config below:
easy_config = {
    "load_dir"        : 'instances/train_10_n60_m60',
    "idx_list"        : list(range(5)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

# Hard Setup: Use the following environment settings. We will evaluate your agent with the same hard config below:
hard_config = {
    "load_dir"        : 'instances/train_100_n60_m60',
    "idx_list"        : list(range(99)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}




In [43]:
from torch._C import dtype
def discounted_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r, dtype=float)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return list(discounted_r)

In [44]:
env = make_multiple_env(**easy_config) 
s = env.reset()   # samples a RANDOM INSTANCE every time env.reset() is called


loading training instances, dir instances/train_10_n60_m60 idx 0
loading training instances, dir instances/train_10_n60_m60 idx 1
loading training instances, dir instances/train_10_n60_m60 idx 2
loading training instances, dir instances/train_10_n60_m60 idx 3
loading training instances, dir instances/train_10_n60_m60 idx 4


In [45]:
from sklearn.preprocessing import normalize
A, b, c0, cuts_a, cuts_b = s
concat = np.hstack((s[0], np.expand_dims(s[1], axis=1)))
concat
# np.linalg.norm(concat, axis=0)

array([[  0,   1,   3, ...,   3,   3, 577],
       [  1,   4,   4, ...,   0,   2, 588],
       [  0,   1,   1, ...,   0,   1, 541],
       ...,
       [  0,   1,   1, ...,   1,   4, 578],
       [  3,   1,   1, ...,   4,   0, 599],
       [  2,   3,   4, ...,   2,   2, 562]])

In [46]:
np.expand_dims(s[1], axis=1)

array([[577],
       [588],
       [541],
       [566],
       [563],
       [586],
       [589],
       [555],
       [585],
       [589],
       [543],
       [549],
       [574],
       [584],
       [554],
       [543],
       [555],
       [553],
       [552],
       [559],
       [547],
       [588],
       [582],
       [576],
       [555],
       [590],
       [548],
       [587],
       [563],
       [551],
       [547],
       [576],
       [576],
       [581],
       [563],
       [589],
       [565],
       [553],
       [562],
       [587],
       [554],
       [564],
       [573],
       [546],
       [561],
       [551],
       [570],
       [547],
       [540],
       [594],
       [578],
       [570],
       [562],
       [544],
       [557],
       [598],
       [579],
       [578],
       [599],
       [562]])

In [47]:
np.hstack((s[0], np.expand_dims(s[1], axis=1)))

array([[  0,   1,   3, ...,   3,   3, 577],
       [  1,   4,   4, ...,   0,   2, 588],
       [  0,   1,   1, ...,   0,   1, 541],
       ...,
       [  0,   1,   1, ...,   1,   4, 578],
       [  3,   1,   1, ...,   4,   0, 599],
       [  2,   3,   4, ...,   2,   2, 562]])

In [48]:
a = np.array([
              [1,2,3,4],
              [3,3,3,3],
              [2,2,2,2],
              [1,1,1,1]
])
print(a.mean(axis=0, keepdims=True))
a = a - a.mean(axis=0, keepdims=True)
a

[[1.75 2.   2.25 2.5 ]]


array([[-0.75,  0.  ,  0.75,  1.5 ],
       [ 1.25,  1.  ,  0.75,  0.5 ],
       [ 0.25,  0.  , -0.25, -0.5 ],
       [-0.75, -1.  , -1.25, -1.5 ]])

In [49]:
empty = [np.array([1,2]), np.array([3,4])]
a = []
b = a + empty + empty

b

[array([1, 2]), array([3, 4]), array([1, 2]), array([3, 4])]

In [50]:
empty = [np.array([10,20])]
b

[array([1, 2]), array([3, 4]), array([1, 2]), array([3, 4])]

In [51]:
%%wandb
# create env
env = make_multiple_env(**easy_config) 
# Parameter initialization
numtrajs = 4  # num of trajecories from the current policy to collect in each iteration
lr_pg = 1e-2  # learning rate for PG
attention_size = 10
iterations = 25
discount_gamma = .99

# Network initialize
policy = Policy(lr_pg, 61, attention_size, 0.5)

#To record training reward for logging and plotting purposes
rrecord = []

VAL_ALL = []  # Monte carlo value predictions of ALL trajectories (to compute baseline, and policy gradient) => 2 d list (numtrajs, #steps per traj)

# For every training iteration, we roll out 5 random instances using the policy network
# collect observation and action matrices from these 5 roll-outs, consider this a BATCH
# train the network using this BATCH
for ite in range(iterations):

    print("==========================================================================================================")
    print("Outer iteration no", ite)

    # To record traectories generated from current policy
    OBS_MAT = []  # observations: [obs_matrices_batch_traj1, obs_matrices_batch_traj2, ....]
    ACT_MAT = []  # actions
    ADS = []  # advantages (to compute policy gradient)
    VAL = []  # Monte carlo value predictions (to compute baseline, and policy gradient) => 2 d list (numtrajs, #steps per traj)
    traj_returns = []  # a list of 5 numbers, each number represents the RETURN of that roll-out
    actions = []
    
    # collect some trajectories
    for num in range(numtrajs):
        print("-------------------------------------------------------------------------------------------")
        print("Running trajectories:", num)
        # Initialize a list of obs_matrices and act_matrices, to store all the obs_matrix and act_matrix in the trajectory
        obs_matrices = []  # states: [obs_matrix_state1, obs_matrix_state2, ...]
        act_matrices = []  # actions matrices
        
        # this is used to collect all the immedaite rewards in this trajectory
        rews = []  # instant rewards
        rel_rews = []

        # gym loop
        s = env.reset()   # samples a RANDOM INSTANCE every time env.reset() is called
        done = False

        # TODO: wandb logging -> what reward average should we log??
        t = 0 # TODO: how is this used?
        repisode = 0  # TODO: how is this used?

        inner_step = 1
        

        # roll out ONE policy
        while not done:

            # TODO compute the running average RETURN as basline
            A, b, c0, cuts_a, cuts_b = s

            # choose the action according to the model output probabilities

            # Concat [A,b] and [cuts_a, cuts_b]
            assert A.shape[0] == b.shape[0]
            assert cuts_a.shape[0] == cuts_b.shape[0]

            obs_matrix = np.hstack((A, np.expand_dims(b, axis=1)))
            act_matrix = np.hstack((cuts_a, np.expand_dims(cuts_b, axis=1)))

            assert obs_matrix.shape == (A.shape[0], A.shape[1]+1)
            assert act_matrix.shape == (cuts_a.shape[0], cuts_a.shape[1]+1)

            # Normalize on a row (MIGHT NOT NEED THIS)
            
            # The reason we want to normalize a row: we want the numeric space of the model input to be just between 0, 1
            # Right now I'm normalizing such that the largest number has value 1 --> each row divided by the largest num in that row
            #   => This would result in b vector always be 1 (does it make sense?)
            # another option is to normalize such that the SUM of the row is 1
            #   => I think this makes more sense, consider this differentiates the max among datapoints
            #   => According to prof in OH this might cause some information loss
            
            obs_matrix = obs_matrix / obs_matrix.max(axis=1, keepdims=True) * 100
            act_matrix = act_matrix / act_matrix.max(axis=1, keepdims=True) * 100
            
            # print("DEBUGGING: obs_matrix.shape = ", obs_matrix.shape)
            # print("DEBUGGING: act_matrix.shape = ", act_matrix.shape)

            action_prob = policy.predict_prob(obs_matrix, act_matrix)
            action = random.choices(range(0, len(cuts_b)), action_prob) # this returns a list
            actions.append(action)

            if  inner_step %10 == 0:
                # TODO: print the logits as well as well.
                print("DEBUGGING: the action_prob is:", action_prob)
                print("DEBUGGING: the actual action to take is:", action)
                logits_dbg = policy.compute_one_step_logits(obs_matrix, act_matrix)
                print("DEBUGGING: logits looks like:", logits_dbg)



            # take the action in the environment
            # TODO: why does the environment.step function takes in a list?
            # TODO: remember to go in the environment to change the returned r to the NORMALIZED r!!
            s, r, done, _ = env.step(action)
            
            # Record the observed immediate reward & observed matrices along the trajectory
            abs_reward, rel_reward = r
            # print("DEBUGGING: rel_reward looks like:", rel_reward)
            rews.append(abs_reward) # rews = list(len of trajectory)
            rel_rews.append(rel_reward * 10 * inner_step)
            obs_matrices.append(obs_matrix)
            act_matrices.append(act_matrix)

            inner_step += 1

            # TODO: do we need to also record the one step of observation and action where the environment terminates?
            #   ==> RN I'm thinking maybe don't need to
        print("-------------------------------------------------------------------------------------------")
        #Below is for logging training performance
        print("DEBUGGING: the total abs reward of the trajectory =", np.sum(rews), "and immediate abs rewards look like:", rews)
        print("DEBUGGING: the total relative reward of the trajectory =", np.sum(rel_rews), "and immediate relative rewards look like:", rel_rews)

        rrecord.append(np.sum(rews))

        # After the policy roll out for this trajectory,
        # compute the monte-carlo RETURN of this trajectory (i.e. discounted sum of rewards), add to big list
        # TODO: one of the next steps could be: to make the basline state-dependent
        v_hat = discounted_rewards(rel_rews, discount_gamma) # This is a list
        traj_returns.append(v_hat[0])
        # OBS_MAT.append(np.concatenate(obs_matrices, axis=1))
        # ACT_MAT.append(np.concatenate(act_matrices, aixs=1))
        VAL.append(v_hat) # VAL -> 2d list
        VAL_ALL.append(v_hat) # VAL_ALL -> 2d list
        OBS_MAT += obs_matrices
        ACT_MAT += act_matrices

        # TODO: do I need to specify batchsize somewhere?

    print("+++++++++++++++++++ The policy roll-out has finished! ++++++++++++++++++++++++++++++++")
    
    # After collecting 5 (or however many) trajectories,
    print("DEBUGGING: OBS_MAT has %d number of matrices" % len(OBS_MAT))
    print("DEBUGGING: ACT_MAT has %d number of matrices" % len(ACT_MAT))
    print("DEBUGGING: VAL looks like:", VAL)
    # print("DEBUGGING: OBS_MAT looks like:", OBS_MAT)
    # print("DEBUGGING: ACT_MAT looks like:", ACT_MAT)
    print("DEBUGGING: traj_returns =", traj_returns)
    print("DEBUGGING: actions =", actions)
    print("DEBUGGING: actions length =", len(actions))

    ## For debugging purposes, let's look at what does the model output
    print("DEBUGGING: what does the model output in this round of roll-out?")
    obs_attention_dbg = policy.model(torch.FloatTensor(obs_matrix))
    act_attention_dbg = policy.model(torch.FloatTensor(act_matrix))
    logits_dbg = policy.compute_one_step_logits(obs_matrix, act_matrix)
    print("DEBUGGING: obs_attention looks like:", obs_attention_dbg)
    print("DEBUGGING: act_attention looks like:", act_attention_dbg)
    print("DEBUGGING: logits looks like:", logits_dbg)


    assert len(traj_returns) == numtrajs
    VAL = np.array(VAL)
    VAL_ALL_np = np.array(VAL_ALL)
    # 1. calculate the baseline: average return of the trajectories
    # TODO: potentially can make this into *running average*, take into account of all the previous trajectories as well
    # TODO: potentially make the baseline the average *VALUE* of every *state*, 
    #       but I'm not sure whether that "state" is useful in this concept, since the first *step*
    #       doesn't really mean the same thing among instances
    #       I think prof says it makes sense in the OH, also it would make more sense if you engineered the reward
    baseline = np.mean(traj_returns)
    baseline_2 = VAL.mean(axis=0, keepdims=True) # This is NOT a good baseline, this is only the mean of that episode, NOT running mean
    baseline_3 = VAL_ALL_np.mean(axis=0, keepdims=True)
    # assert baseline_2.shape == VAL.shape

    # 2. Update the policy
    ADS = (VAL - baseline_2).flatten()
    print("DEBUGGING: baseline2 looks like:", baseline_2)
    print("DEBUGGING: baseline2 looks like:", baseline)
    print("DEBUGGING: ADS looks like:", ADS)


    # Train the agent using the batch
    # obs_batch = np.concatenate(OBS_MAT)
    # act_batch = np.concatenate(ACT_MAT)

    assert ADS.shape[0] == len(actions)

    # scaling up the rewards to artificially make bigger loss, improve learning

    loss = policy.train(OBS_MAT, ACT_MAT, actions, ADS)

    fixedWindow=100
    movingAverage=0
    if len(rrecord) >= fixedWindow:
        movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])

    # TODO: wandb logging
    wandb.log({ "training reward" : rrecord[-1], "training reward moving average" : movingAverage, "training loss": loss})
    #make sure to use the correct tag in wandb.init in the initialization on top


Streaming output truncated to the last 5000 lines.
        [ 0.0339,  0.0040,  0.0332,  ...,  0.0587, -0.0685, -0.1578],
        [ 0.0249,  0.0030,  0.0244,  ...,  0.0432, -0.0505, -0.1170]])
   Last layer:
tensor([[-0.0589, -0.0158,  0.0000, -0.0551,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0578,
          0.0000, -0.0589, -0.0665, -0.0109],
        [-0.0485, -0.0123,  0.0000, -0.0448,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0462,
          0.0000, -0.0468, -0.0526, -0.0084],
        [-0.0515, -0.0141,  0.0000, -0.0485,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0511,
          0.0000, -0.0523, -0.0590, -0.0098],
        [ 0.0148,  0.0039,  0.0000,  0.0138,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0143,
       

In [52]:
run.finish()

training loss,▂▂▁▂▂▂▁▂▁▂▂▂▆▂▁▂▁▁▂▇█▄▃▂▁
training reward,▁▁▆▁▆▆▂▆▁█▆▂▆█▁▆▂▃▆▁▆▁█▁▂
training reward moving average,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
training loss,-0.22443
training reward,0.17259
training reward moving average,0.49766


In [53]:
a = [   54246.9141, 27220988.0000,  3948314.5000,  3902082.5000,
        27219886.0000,  3816622.7500, 27175632.0000,  3976903.5000,
         3977309.0000,  3961905.7500,  3904790.7500, 23420360.0000,
        27218672.0000, 27206276.0000, 27190054.0000,  3976984.7500,
         3931598.0000,  3963138.5000, 19419398.0000,  3931559.5000,
         7862683.0000, 19299310.0000,  3915878.7500, 15562425.0000,
        27141808.0000, 27323768.0000, 11622510.0000, 15451909.0000,
        23389744.0000,  7781165.5000, 11871555.0000, 11657742.0000,
        27473456.0000, 11592561.0000,  4054315.2500,  7718911.0000,
        27136936.0000,  3914171.2500, 23255876.0000, 15599427.0000,
        19661078.0000,  3915166.7500,  7554840.5000, 27196042.0000,
        11791202.0000, 23477022.0000, 15708443.0000, 23297932.0000,
        23511102.0000,  7689591.5000, 27405178.0000,  4021836.7500,
        15558291.0000,  3948567.5000,  7915618.5000,  3864714.5000,
         3995934.2500, 19536616.0000,  3890406.7500, 27284138.0000,
        11731948.0000, 23232848.0000, 19200360.0000, 22949674.0000,
        26724900.0000,  3268551.2500, 22647966.0000, 25925040.0000,
        25380838.0000,  5669201.5000,  7810961.0000, 13092715.0000,
        14364957.0000, 15622157.0000, 27473452.0000]

In [54]:
a_t = torch.FloatTensor(a)
print(a_t)
b_t = a_t /1000000
print(b_t)
print(torch.nn.functional.softmax(b_t, dim=0))
b_t = a_t / a_t.max()
print(b_t)
torch.nn.functional.softmax(b_t, dim=0)

tensor([   54246.9141, 27220988.0000,  3948314.5000,  3902082.5000,
        27219886.0000,  3816622.7500, 27175632.0000,  3976903.5000,
         3977309.0000,  3961905.7500,  3904790.7500, 23420360.0000,
        27218672.0000, 27206276.0000, 27190054.0000,  3976984.7500,
         3931598.0000,  3963138.5000, 19419398.0000,  3931559.5000,
         7862683.0000, 19299310.0000,  3915878.7500, 15562425.0000,
        27141808.0000, 27323768.0000, 11622510.0000, 15451909.0000,
        23389744.0000,  7781165.5000, 11871555.0000, 11657742.0000,
        27473456.0000, 11592561.0000,  4054315.2500,  7718911.0000,
        27136936.0000,  3914171.2500, 23255876.0000, 15599427.0000,
        19661078.0000,  3915166.7500,  7554840.5000, 27196042.0000,
        11791202.0000, 23477022.0000, 15708443.0000, 23297932.0000,
        23511102.0000,  7689591.5000, 27405178.0000,  4021836.7500,
        15558291.0000,  3948567.5000,  7915618.5000,  3864714.5000,
         3995934.2500, 19536616.0000,  3890406.7

tensor([0.0074, 0.0199, 0.0085, 0.0085, 0.0199, 0.0085, 0.0199, 0.0085, 0.0085,
        0.0085, 0.0085, 0.0173, 0.0199, 0.0199, 0.0199, 0.0085, 0.0085, 0.0085,
        0.0150, 0.0085, 0.0098, 0.0149, 0.0085, 0.0130, 0.0198, 0.0200, 0.0113,
        0.0130, 0.0173, 0.0098, 0.0114, 0.0113, 0.0201, 0.0113, 0.0086, 0.0098,
        0.0198, 0.0085, 0.0172, 0.0130, 0.0151, 0.0085, 0.0097, 0.0199, 0.0113,
        0.0174, 0.0131, 0.0172, 0.0174, 0.0098, 0.0200, 0.0085, 0.0130, 0.0085,
        0.0099, 0.0085, 0.0085, 0.0150, 0.0085, 0.0199, 0.0113, 0.0172, 0.0149,
        0.0170, 0.0195, 0.0083, 0.0168, 0.0190, 0.0186, 0.0091, 0.0098, 0.0119,
        0.0125, 0.0130, 0.0201])

In [55]:
s[0][0]

array([2., 0., 1., 3., 0., 0., 0., 3., 2., 3., 1., 1., 2., 0., 4., 4., 0.,
       2., 1., 2., 2., 2., 4., 1., 3., 2., 0., 1., 2., 0., 3., 0., 3., 1.,
       3., 0., 4., 1., 4., 4., 0., 0., 1., 2., 4., 0., 0., 1., 1., 1., 2.,
       3., 4., 4., 3., 3., 0., 0., 0., 0.])

In [56]:
s[1][-1]

20655680.0

In [57]:
s[-2] / np.max(s[-2], axis=1, keepdims=True)

array([[0.54545455, 0.63636364, 0.81818182, ..., 0.63636364, 0.81818182,
        0.63636364],
       [0.59204253, 0.72272276, 0.80233403, ..., 0.71290849, 0.71459982,
        0.58222368],
       [0.59206416, 0.72271493, 0.80230557, ..., 0.71290297, 0.71451933,
        0.58222376],
       ...,
       [0.59207576, 0.72270644, 0.80232062, ..., 0.71292965, 0.71457005,
        0.58218961],
       [0.59205913, 0.72271276, 0.80231302, ..., 0.71291558, 0.71456072,
        0.58223248],
       [0.59204855, 0.72270337, 0.80230828, ..., 0.7129393 , 0.7145267 ,
        0.58223474]])

In [58]:
len(s[-2]), len(s[0])

(110, 110)

In [59]:
a = np.array([0.0057, 0.0235, 0.0144, 0.0181, 0.0181, 0.0123, 0.0293, 0.0095, 0.0122,
        0.0146, 0.0152, 0.0095, 0.0184, 0.0151, 0.0143, 0.0095, 0.0220, 0.0120,
        0.0209, 0.0226, 0.0142, 0.0124, 0.0145, 0.0133, 0.0218, 0.0146, 0.0098,
        0.0186, 0.0146, 0.0126, 0.0182, 0.0204, 0.0155, 0.0180, 0.0098, 0.0184,
        0.0248, 0.0150, 0.0215, 0.0114, 0.0186, 0.0175, 0.0141, 0.0199, 0.0250,
        0.0178, 0.0310, 0.0170, 0.0153, 0.0204, 0.0121, 0.0247, 0.0187, 0.0197,
        0.0130, 0.0127, 0.0240, 0.0189, 0.0104, 0.0125])
a.sum()

0.9999

In [60]:
env.env_now.env.remain_gap

AttributeError: ignored

In [ ]:
env.env_now.env.oldobj

In [ ]:
env.env_now.env.newobj


In [ ]:
env.env_now.env.ip_obj

# OH notes:

baseline is what you use to interprete the reward

don't use nn, use "mean" of rewards in this episode (I think so)

Maybe look in to the instances, and look at how your agent is solving them

Cutting off early, but only after solved some LP's
- counterfactual exploration
  - more than to do it more random
- all prob entirely the same
- activation function ?

not learning? 

reward shaping?
- someone: amplify the reward (shouldn't, since every step wil GIVE a reward), compare it to other POSSIBLE states
- pre-trained on the instances and pre-trained the baseline?
- get the max reward from the LP solver (isn't this cheating lol)
- recalcluate the max-gap-to-go (lol remaining max gap) every step
- go in the environment to make if return the shaped new-gap reward
- the original reward doesn't help you across LPs?
- moving average of *returns* from all *previous* episodes
  - return is the discounted sum of the reward
- advantage? Q - running averaged RETURN (but different states are mixed in, how do you deal with that)
- Think about what's wrong with this base line, and write it down
  - something about the states


differences between LPs

mode: 
- standardize the constraints, and the b vector by itself
  - do you normalize by row or columns?
  - He thinks it's more sense to normalize by rows
  - Normalize it twice? LOL
- or a normalization layer\


When doing softmax you can use a lamda (parameter) to mitigate large score difference
- softmax comes with a scalar parameter

Professor didn't think normalizing the input is necessary & and it shouldn't make a difference anyways (but come numerical value can be lost after normalization)

iteration: 

plot random policy together, (as a bseline to see if your model is really learning)

For baseline, Prof is suggesting Q network can work too.


